# 2017年10月27日：
## 尝试利用脸部和眼部的bottleneck的特征来实现回归问题


In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.python.platform import gfile
import os
import time
import cv2
import pickle
import interface
import matplotlib.pyplot as plt
from Vgg_class import Vgg_19
sess=tf.InteractiveSession()
#face_img=np.load('face_10k.npy')

In [2]:
Input_dim=7*7*512

In [23]:
def drew_face_eye(img,show_reg=True):
        '''
        检测用户当前环境是否能够比较好的识别面部特征
        辨识出脸部以及眼部，标记出来
        :param img:
        :return:
        '''
        face_cascade = cv2.CascadeClassifier(r'D:\Proj_DL\Code\Proj_EyeTraker\haarcascade_frontalface_default.xml')
        eye_cascade = cv2.CascadeClassifier(r'D:\Proj_DL\Code\Proj_EyeTraker\haarcascade_eye.xml')
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3,5)
        #确保画面中只有一个人脸识别出来
        if(len(faces)!=1):
            return None
        for (x,y,w,h) in faces:
            roi_color = img[y:y+h, x:x+w]
            roi_gray = gray[y:y+h, x:x+w]
#             #保存脸部图像
            face_mat=roi_color
            eyes = eye_cascade.detectMultiScale(roi_gray,scaleFactor=1.2, minNeighbors=7, minSize=(29,29),
                                            flags=cv2.CASCADE_SCALE_IMAGE)
            #眼睛检测 ,对于识别比较差的情况舍弃
            if len(eyes)!=2:
                return None
            if eyes[0][0]>eyes[1][0]:
                ex=eyes[1][0]
                W=eyes[0][0]-eyes[1][0]+eyes[0][2]
            else:
                ex=eyes[0][0]
                W=eyes[1][0]-eyes[0][0]+eyes[1][2]

            if eyes[0][1]>eyes[1][1]:
                ey=eyes[1][1]
                H=eyes[0][1]-eyes[1][1]+eyes[0][3]
            else:
                ey=eyes[0][1]
                H=eyes[1][1]-eyes[0][1]+eyes[1][3]
            eye_mat=roi_color[ey:ey+H,ex:ex+W]
        return  (img,face_mat,eye_mat)
file_root=r'D:/Proj_DL/Code/Proj_EyeTraker/Proj_iTraker/Transfer_iTraker/img4X4_val/'
img_names=os.listdir(file_root)
#打乱顺序
index=np.arange(len(img_names))
np.random.shuffle(index)
img_names=list(np.array(img_names)[index])
print(len(img_names))

1184


### 加载 inception 模型

In [4]:
#jpg 二进制输入
JEPG_INPUT='DecodeJpeg/contents:0'
#矩阵输入
MAT_INPUT='ExpandDims:0'

#bottleneck 输出(1,2048)
BOTTLE_NECK='pool_3/_reshape:0'
#最后一个池化层的输出(1,8,8,2048)
MIX10='mixed_10/join:0'
#mix9输出（1,8,8,2048)
MIX9='mixed_9/join:0'
#mix8输出(1,8,8,1280)
MIX8='mixed_8/join:0'
#mix8输出(1,17,17,768)
MIX7='mixed_7/join:0'
#mixed输出(1,35,35,256)
MIX0='mixed/join:0'

MODEL_DIR='inceptionv3_model/tensorflow_inception_graph.pb'
with gfile.FastGFile(MODEL_DIR,'rb') as f:
    graph_def=tf.GraphDef()
    graph_def.ParseFromString(f.read())
feature_bottleneck,jpeg_tensor=tf.import_graph_def(graph_def,return_elements=[MIX10,MAT_INPUT])

In [ ]:
y_cordinate=[]
face_matrix=[]
eye_matrix=[]
fail_counter=0
s_time=time.time()
file_index=0
if not os.path.exists('Feature_File_bottleneck2'):
    print('make file fold:Feature_File_bottleneck!')
    os.mkdir('Feature_File_bottleneck2')
for i,f in enumerate(img_names):
    lable_id=int(f[:-4].split('_')[-1])
    img=cv2.imread(os.path.join(file_root,f))
    face_op=drew_face_eye(img)
    if face_op == None:
        fail_counter+=1
        continue
    _,f,e=face_op
    f=f.reshape((1,f.shape[0],f.shape[1],f.shape[2]))
    e=e.reshape((1,e.shape[0],e.shape[1],e.shape[2]))
    fea_f=sess.run(feature_bottleneck,{jpeg_tensor:f}).reshape((1,8,8,2048))
    fea_e=sess.run(feature_bottleneck,{jpeg_tensor:e}).reshape((1,8,8,2048))
    lable_=[0]*16
    lable_[lable_id]=1
    y_cordinate.append(lable_)
    face_matrix.append(fea_f.astype('float32'))
    eye_matrix.append(fea_e.astype('float32'))
    if i%100==0:
        print('pic:{},fail count:{},run time:{}'.format(i,fail_counter,time.time()-s_time))
       # s_time=time.time()
    if (i%50000==0)and (i !=0):
        file_index+=1
        file_name='feature50k_bottle_'+str(file_index)+r'_.pkl'
        fp=open(os.path.join(r'Feature_File_bottleneck2/',file_name),'wb')
        face_matrix=np.array(face_matrix).astype('float32')
        eye_matrix=np.array(eye_matrix).astype('float32')
        y_cordinate=np.array(y_cordinate).astype('uint8')
        pickle.dump(obj={'feature_face':face_matrix,'feature_eye':eye_matrix,'label':y_cordinate},file=fp)
        fp.close()
        y_cordinate=[]
        face_matrix=[]
        eye_matrix=[]
        print('file save')
        #break

In [ ]:
face_matrix.shape

In [ ]:
face_matrix=np.array(face_matrix).astype('float32').reshape((-1,8,8,2048))
eye_matrix=np.array(eye_matrix).astype('float32').reshape((-1,8,8,2048))
y_cordinate=np.array(y_cordinate).astype('uint8')
fp=open('fea1k_mix10.pkl','wb')
pickle.dump(file=fp,obj={'feature_face':face_matrix,'feature_eye':eye_matrix,'label':y_cordinate})
fp.close()

In [ ]:
d=np.load('fea908_bottleneck.pkl')
fea_mat=[]
y_xy=[]
for i in range(0,d['label'].shape[0]):
   
    f_=np.concatenate((d['feature_eye'][i],d['feature_face'][i]),axis=0)
    #just eyes
#     f_=d['feature_eye'][i]
    #just face
#     f_=d['feature_face'][i]
#     id_=np.argmax(d['label'][i])
#     x_,y_=id_%4,id_//4
#     y_xy.append((x_,y_))
    fea_mat.append(f_)
fea_mat=np.asarray(fea_mat)
y_xy=np.asarray(d['label']).reshape((-1,16))

In [ ]:
fea_mat.shape


# *训练CNN进行分类*


In [ ]:

#输入 X: (N,4096)
def model_mlp(x_input,traing):
    trainable_=True
    drop_rate=0.1
    with tf.variable_scope('fc_4'):
        w=tf.get_variable('weight',shape=[Input_dim,2048],initializer=tf.contrib.layers.xavier_initializer())
        b=tf.get_variable('biases',initializer=tf.constant(0.1,shape=[2048,]),trainable=trainable_)
        y_l1=tf.matmul(x_input,w)+b
        y_l1=tf.layers.batch_normalization(y_l1,training=traing)
        y_l1=tf.nn.relu(y_l1)
    #fc5 (N,2048)->(N,1024)
    with tf.variable_scope('fc_5'):
        y_l1=tf.nn.dropout(y_l1,drop_rate)
        w=tf.get_variable('weight',shape=[2048,1024],initializer=tf.contrib.layers.xavier_initializer())
        b=tf.get_variable('biases',initializer=tf.constant(0.1,shape=[1024,]),trainable=trainable_)
        y_l2=tf.matmul(y_l1,w)+b
        y_l2=tf.layers.batch_normalization(y_l2,training=traing)
        y_l2=tf.nn.relu(y_l2)
    #fc6 (N,1024)->(N,16)
    with tf.variable_scope('fc_6'):
        y_l2=tf.nn.dropout(y_l2,drop_rate)
        w=tf.get_variable('weight',shape=[1024,16],initializer=tf.contrib.layers.xavier_initializer())
        b=tf.get_variable('biases',initializer=tf.constant(0.1,shape=[16,]))
        y_l3=tf.matmul(y_l2,w)+b
    return y_l3

def ReloadData():
    index=np.random.choice(range(1,10),2,replace=False)
    
    file_name=r'Feature_File/feature50k_pool3_'+str(index[0])+'_.pkl'
    d=np.load(file_name)
    fea_data=d['feature'].reshape((d['feature'].shape[0],d['feature'].shape[2],d['feature'].shape[3],d['feature'].shape[4],))
    fea_label=d['label']
    file_name=r'Feature_File/feature50k_pool3_'+str(index[1])+'_.pkl'
    d=np.load(file_name)
    f_=d['feature'].reshape((d['feature'].shape[0],d['feature'].shape[2],d['feature'].shape[3],d['feature'].shape[4],))
    fea_data=np.concatenate((fea_data,f_),axis=0)
    fea_label=np.concatenate((fea_label,d['label']),axis=0)
    #将id 装换成 坐标
    fea_label=conver_id2xy(fea_label)
    return fea_data,fea_label

In [ ]:
#输入 X: (N,8,8,2048)
def model_cnn(x_input,traing):
    trainable_=True
    drop_rate=0.1
    regularizer=tf.contrib.layers.l2_regularizer(1e-2)
    #conv1 1*1/1 (8,8,2048)->(8,8,1024)
    with tf.variable_scope('conv_1'):
        w=tf.get_variable('weight',shape=[1,1,2048,256],initializer=tf.contrib.layers.xavier_initializer(),trainable=trainable_)
        b=tf.get_variable('biases',initializer=tf.constant(0.1,shape=[256,]),trainable=trainable_)
        tf.add_to_collection('loss_w',regularizer(w))
        conv_=tf.nn.conv2d(x_input,w,[1,1,1,1],'VALID')+b
        conv_=tf.layers.batch_normalization(conv_,training=traing)
        conv_=tf.nn.relu(conv_)
        
    #conv2 3*3/2 (8,8,1024)->(6,6,1024)
    with tf.variable_scope('conv_2'):
        w=tf.get_variable('weight',shape=[3,3,256,128],initializer=tf.contrib.layers.xavier_initializer(),trainable=trainable_)
        b=tf.get_variable('biases',initializer=tf.constant(0.1,shape=[128,]),trainable=trainable_)
        tf.add_to_collection('loss_w',regularizer(w))
        conv_=tf.nn.conv2d(conv_,w,[1,1,1,1],'VALID')+b
        conv_=tf.layers.batch_normalization(conv_,training=traing)
        conv_=tf.nn.relu(conv_)
    #conv3 3*3/1 (6,6,1024)->(4,4,512)
    with tf.variable_scope('conv_3'):
        w=tf.get_variable('weight',shape=[3,3,128,64],initializer=tf.contrib.layers.xavier_initializer(),trainable=trainable_)
        b=tf.get_variable('biases',initializer=tf.constant(0.1,shape=[64,]),trainable=trainable_)
        tf.add_to_collection('loss_w',regularizer(w))
        conv_=tf.nn.conv2d(conv_,w,[1,1,1,1],'VALID')+b
        conv_=tf.layers.batch_normalization(conv_,training=traing)
        conv_=tf.nn.relu(conv_)
     #conv4 3*3/1 (4,4,512)->(2,2,512)
    with tf.variable_scope('conv_4'):
        w=tf.get_variable('weight',shape=[3,3,64,32],initializer=tf.contrib.layers.xavier_initializer(),trainable=trainable_)
        b=tf.get_variable('biases',initializer=tf.constant(0.1,shape=[32,]),trainable=trainable_)
        tf.add_to_collection('loss_w',regularizer(w))
        conv_=tf.nn.conv2d(conv_,w,[1,1,1,1],'VALID')+b
        conv_=tf.layers.batch_normalization(conv_,training=traing)
        conv_=tf.nn.relu(conv_)
      
    #fc4 (N,2*2*768)->(N,2048)
    with tf.variable_scope('fc_4'):
        x_flatten=tf.reshape(conv_,shape=[-1,2*2*32]) # N*256
        w=tf.get_variable('weight',shape=[2*2*32,100],initializer=tf.contrib.layers.xavier_initializer(),trainable=trainable_)
        b=tf.get_variable('biases',initializer=tf.constant(0.1,shape=[100,]),trainable=trainable_)
        tf.add_to_collection('loss_w',regularizer(w))
        y_l1=tf.matmul(x_flatten,w)+b
        y_l1=tf.layers.batch_normalization(y_l1,training=traing)
        y_l1=tf.nn.relu(y_l1)
    #fc5 (N,2048)->(N,1024)
    with tf.variable_scope('fc_5'):
        y_l1=tf.nn.dropout(y_l1,drop_rate)
        w=tf.get_variable('weight',shape=[100,100],initializer=tf.contrib.layers.xavier_initializer(),trainable=trainable_)
        b=tf.get_variable('biases',initializer=tf.constant(0.1,shape=[100,]),trainable=trainable_)
        tf.add_to_collection('loss_w',regularizer(w))
        y_l2=tf.matmul(y_l1,w)+b
        y_l2=tf.layers.batch_normalization(y_l2,training=traing)
        y_l2=tf.nn.relu(y_l2)
    #fc6 (N,1024)->(N,16)
    with tf.variable_scope('fc_6'):
        y_l2=tf.nn.dropout(y_l2,drop_rate)
        w=tf.get_variable('weight',shape=[100,16],initializer=tf.contrib.layers.xavier_initializer())
        b=tf.get_variable('biases',initializer=tf.constant(0.1,shape=[16,]))
        tf.add_to_collection('loss_w',regularizer(w))
        y_l3=tf.matmul(y_l2,w)+b
    return y_l3

In [ ]:
d=np.load('vgg19_feature.pkl')
fea_mat=d['feature_eye']
y_xy=np.asarray(d['label'])
del d

In [ ]:
X=tf.placeholder(tf.float32,[None,7*7*512])
Y=tf.placeholder(tf.float32,[None,16])
TRAINGING=tf.placeholder(tf.bool)
#
y_socre=model_mlp(X,TRAINGING)

# Loss=(tf.losses.mean_squared_error(Y,y_socre))
# tf.summary.scalar('loss',Loss)
# ERR=tf.reduce_mean(tf.sqrt(tf.reduce_sum(tf.squared_difference(y_socre, Y), axis=1)))
# tf.summary.scalar('err',ERR)

Loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_socre,labels=Y))
ACC_C=tf.equal(tf.arg_max(y_socre,1),tf.arg_max(Y,1))
ACC=tf.reduce_mean(tf.cast(ACC_C,tf.float32))
tf.summary.scalar('loss',Loss)
tf.summary.scalar('Accuracy',ACC)

TRAIN=tf.train.AdamOptimizer(1e-3).minimize(Loss)
#ACC
BN_OPS=tf.get_collection(tf.GraphKeys.UPDATE_OPS)
writer_tr=tf.summary.FileWriter('./mylog/cnn_mix8/train')
writer_te=tf.summary.FileWriter('./mylog/cnn_mix8/test')

In [ ]:
# fea_mat=np.asarray(fea_mat).reshape((-1,4096))
# y_xy=np.asarray(y_xy).reshape((-1,2))
index=np.arange(fea_mat.shape[0])
np.random.shuffle(index)
tr_index=index[:int(fea_mat.shape[0]*0.8)]
te_index=index[int(fea_mat.shape[0]*0.8):]
tr_data,tr_label=fea_mat[tr_index],y_xy[tr_index]
te_data,te_label=fea_mat[te_index],y_xy[te_index]

In [ ]:
te_data.shape,tr_data.shape

In [ ]:
merge=tf.summary.merge_all()
init=sess.run(tf.global_variables_initializer())

In [ ]:
ERR=ACC
#100 大概1min
acc_his=[]
loss_his=[]
vacc_his=[]
vloss_his=[]
best_te=0
for i in range(1,1000*1000):
    mask=np.random.choice(range(tr_data.shape[0]),128,replace=False)
    x_,y_=tr_data[mask],tr_label[mask]
    sess.run([TRAIN,BN_OPS],{X:x_,Y:y_,TRAINGING:True })
                  
    if i%5==0:
#         mask=np.random.choice(range(val_fea.shape[0]),128,replace=False)
#         x_,y_=val_fea[mask],val_y[mask]
        loss_,acc_,m_,_=sess.run([Loss,ERR,merge,BN_OPS],{X:x_,Y:y_,TRAINGING:False})
#         acc_his.append(acc_)
#             loss_his.append(loss_)
        writer_tr.add_summary(m_,i)
        print('epoch:{},loss:{},err:{}'.format(i,loss_,acc_))
    if i%10==0:
        mask=np.random.choice(range(te_data.shape[0]),128,replace=False)
        x_,y_=te_data[mask],te_label[mask]
        loss_,acc_,m_,_=sess.run([Loss,ERR,merge,BN_OPS],{X:x_,Y:y_,TRAINGING:True})
#         vacc_his.append(acc_)
        best_te=max(acc_,best_te)
#         vloss_his.append(loss_)
        writer_te.add_summary(m_,i)
        print('--epoch:{},loss:{},err:{},best acc:{}'.format(i,loss_,acc_,best_te))
                 

In [ ]:
plt.plot(vacc_his,label='val')
plt.plot(acc_his,label='train')
plt.legend()
plt.show()

In [ ]:
np.mean(vacc_his[-100:]),np.mean(acc_his[-100:])

In [ ]:
d=np.load('Feature_File_bottleneck2/feature15k_bottle_1_.pkl')
val_fea=[]
val_y=[]
for i in range(0,d['label'].shape[0]):
    f_=d['feature_face'][i]
    val_fea.append(f_)
val_fea=np.asarray(val_fea).reshape((-1,Input_dim))
val_y=np.asarray(d['label']).reshape((-1,16))
val_fea-=np.mean(fea_mat,0)

In [ ]:
fname='justface.pkl'
fp=open(fname,'wb')
pickle.dump(file=fp,obj={'acc':acc_his,'loss':loss_his})
fp.close()


In [ ]:
#保存
saver=tf.train.Saver()
saver.save(sess,'./model_save/inception_err/mlpmodelerr045'+'.ckpt',global_step=i)

In [ ]:
#复原

saver=tf.train.Saver()
saver.restore(sess,'./model_save/inception_err/mlpmodelerr045.ckpt-363615')


## 测试

In [ ]:
import cv2
import time

In [ ]:
def drawline(img_,line_w=1,line_color=(0,0,0),wandh_num=4):
        '''
        在图片上格子
        :param line_w: 线宽
        :param line_color: 线颜色
        :param wandh_num:  长宽线的数量
        :return:  无
        '''
        h,w=img_.shape[0],img_.shape[1]
        w_num,h_num=wandh_num,wandh_num
        h_,w_=h//h_num,w//w_num

        # 竖线 (w,h)
        for i in range(1,w_num):
            #print(i)
            cv2.line(img_,(w_*i,0),(w_*i,h),line_color,line_w)
        # 横线
        for i in range(1,h_num):
            cv2.line(img_,(0,h_*i),(w,h_*i),line_color,line_w)
        return img_

def drew_face_eye2(img):
        '''
        检测用户当前环境是否能够比较好的识别面部特征
        辨识出脸部以及眼部，标记出来
        :param img:
        :return:
        '''
        face_cascade = cv2.CascadeClassifier(r'D:\Proj_DL\Code\Proj_EyeTraker\haarcascade_frontalface_default.xml')
        eye_cascade = cv2.CascadeClassifier(r'D:\Proj_DL\Code\Proj_EyeTraker\haarcascade_eye.xml')
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3,5)
        #确保画面中只有一个人脸识别出来
        if(len(faces)!=1):
            return None
        for (x,y,w,h) in faces:
            cv2.rectangle(img,(x-10,y-10),(x+w+10,y+h+10),(255,0,0),2)
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = img[y:y+h, x:x+w]
            #保存脸部图像
            face_mat=roi_color.copy()
            #检测视频中脸部的眼睛，并用vector保存眼睛的坐标、大小（用矩形表示）
            eyes = eye_cascade.detectMultiScale(roi_gray,scaleFactor=1.1, minNeighbors=5, minSize=(27, 27),
                                                flags=cv2.CASCADE_SCALE_IMAGE)
            if len(eyes)!= 2:
                return None
            eye_mat=[]
            for e in eyes:
                xe,ye,we,he=e
                eye_mat.append(face_mat[ye:ye+he,xe:xe+we])
                cv2.rectangle(roi_color,(xe-3,ye-3),(xe+we+3,ye+he+3),(0,0,255),2)
        return  (img,face_mat,eye_mat)
def drawblock(img,line_num,block_id=0,blockcolor=(46,218,255),blockwideth=5,show_rec=True):
        '''
        选定九宫格，在这个格子上填充矩形表示选定这个格子
        :param img_: 图片
        :param block: 九宫格序号 0-15
        :param blockcolor: 矩形框颜色
        :param blockwideth: 框的宽度
        :return:
        '''
        h,w=img.shape[0],img.shape[1]
        w_line,h_line=line_num,line_num
        h_,w_=h//h_line,w//w_line
        cor_h=block_id//line_num
        cor_w=block_id%line_num
        sx,sy=cor_w*w_,cor_h*h_
        
        if show_rec:
        #将整个矩形填充为其他颜色
            img[sy:sy+h_,sx:sx+w_,:]=blockcolor
        else:
            roi_=img[sy:sy+h_,sx:sx+w_]
            cv2.circle(roi_,(roi_.shape[1]//2,roi_.shape[0]//2), 10, (255,128,120), -1)
       
        return img

In [ ]:
tar_img=cv2.imread('test.jpg')
tar_img=drawline(tar_img,wandh_num=4)
tar_img=drawblock(tar_img,line_num=4,block_id=6)
cap=cv2.VideoCapture(0)
_,frame=cap.read()
cap.release()
v_test=tar_img.copy()
tar_img[:480,1280:]=frame
cv2.namedWindow('Capture001',cv2.WINDOW_NORMAL)
cv2.setWindowProperty('Capture001', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
cv2.imshow('Capture001',tar_img)
cv2.waitKey()
cv2.destroyWindow('Capture001')

In [ ]:
#分类
#prob=tf.nn.softmax(y_socre)
#guess=tf.arg_max(prob,1)
#回归

In [ ]:
cap=cv2.VideoCapture(0)
while True:
    ret,fram=cap.read()
    if ret:
        cv2.imshow('cam',fram)
        if cv2.waitKey(30)&0xff==27:
            break
cap.release()
cv2.destroyAllWindows()

In [ ]:
fps=10
fsize_desk=(1920,1080)
# save video
video_d=cv2.VideoWriter('demo_desk.flv',cv2.VideoWriter_fourcc('F','L','V','1'),fps,fsize_desk)
cap=cv2.VideoCapture(0)
s_time=time.time()
tar_img=cv2.imread('test.jpg')
#设置window 为全屏
cv2.namedWindow('Capture001',cv2.WINDOW_NORMAL)
cv2.setWindowProperty('Capture001', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

s_time=time.time()
random_show=np.random.randint(0,16,1)[0]

#每N帧输出平均预测
sum_score=[]
sum_counter=0
block_id=0
ret_counter=0
right_frame_counter=0
while True:
    ret,fram=cap.read()
    if ret:
        ret_counter+=1
        #显示 内窥镜图像
        tar_img=cv2.imread('test.jpg')
        tar_img=drawline(tar_img,wandh_num=4)
        #每30帧换点
        if ret_counter%30==0:
            random_show=np.random.randint(0,16,1)[0]
        tar_img=drawblock(tar_img,line_num=4,block_id=random_show)
        cv2.imshow('Capture001',tar_img)
       
        #预测注视方向
        face_op=drew_face_eye(fram)
        if face_op == None:
            continue
        _,f,e=face_op
        #
        f=f.reshape((1,f.shape[0],f.shape[1],f.shape[2]))
        
        fea_f=sess.run(feature_bottleneck,{jpeg_tensor:f})

        y_guess=sess.run(y_socre,feed_dict={X:fea_f,TRAINGING:False})[0]
        #平均每 N 帧的预测分数
        sum_counter+=1
        sum_score.append(y_guess)
        if sum_counter==10:
            sum_score=np.asarray(sum_score).reshape((-1,16))
            mean_score=np.mean(sum_score,0)
            block_id=np.argmax(mean_score)
            sum_score=[]
            sum_counter=0
           
        #block_id=4*int(y_guess[1])+int(y_guess[0])
       # block_id=np.argmax(y_guess)
        if block_id==random_show:
             right_frame_counter+=1
        tar_img=drawblock(tar_img,line_num=4,block_id=block_id,show_rec=False)
        v_img=tar_img.copy()
        v_img[:480,1280:]=fram
        cv2.imshow('Capture001',tar_img)
        #print(time.time()-s_time)
        s_time=time.time()
        video_d.write(v_img)
        if cv2.waitKey(1)&0xff==27:
            print('accuracy:{}'.format(right_frame_counter/ret_counter))
            print('out')
            break
    if (time.time()-s_time)>60*5:
        print('time out')
        break
video_d.release()
cap.release()
cv2.destroyAllWindows()

### 制作图片，bottleneck的各种组合

In [ ]:
f=np.load('justface.pkl')
e=np.load('justeye.pkl')
fe=np.load('faceeye_concate.pkl')
for i in range(len(fe['acc'])):
    fe['acc'][i]*=100
def fn(a,aname,pic_name,b=None,bname=None,acc=True):
    fig=plt.figure(figsize=(10,5))
    if bname:
        plt.plot(a,label=aname)
        plt.xlabel('iteration')
        if acc:plt.ylabel('accuracy(%)')
        else:plt.ylabel('loss')
        plt.plot(b,label=bname)
        plt.xlabel('iteration')
        if acc:plt.ylabel('accuracy')
        else:plt.ylabel('loss')
        plt.legend()
    else:
        plt.plot(a,label=aname)
        plt.xlabel('iteration')
        if acc:plt.ylabel('accuracy(%)')
        else:plt.ylabel('loss')
        plt.legend()
    plt.show()
    fig.savefig(pic_name)
fn(fe['loss'],'concatenate','concate_loss.png',acc=False)
fig=plt.figure(figsize=(10,5))
plt.plot(fe['acc'],label='concatenate')
plt.plot(f['acc'],label='face')
plt.plot(e['acc'],label='eye')
plt.xlabel('iteration')
plt.ylabel('accuracy(%)')
plt.legend()
plt.show()
fig.savefig('acc_all.png')

## VGG迁移学习

In [ ]:
Images=tf.placeholder(tf.float32,[None,224,224,3])
vgg_model=Vgg_19(model_path=r'D:\Data warehouse\Model\VGG_16&19\vgg19.npy',images=Images,\
                 train_list=[],session=sess,image_batch_size=128)
conv5_feature=vgg_model.pool5
#feature=sess.run(conv5_feature,feed_dict=feed_dict)

In [ ]:
y_cordinate=[]
face_matrix=[]
eye_matrix=[]
fail_counter=0
s_time=time.time()
file_index=0
if not os.path.exists('Feature_File_bottleneck2'):
    print('make file fold:Feature_File_bottleneck!')
    os.mkdir('Feature_File_bottleneck2')
for i,f in enumerate(img_names):
    lable_id=int(f[:-4].split('_')[-1])
    # BGR
    img=cv2.imread(os.path.join(file_root,f))
    face_op=drew_face_eye(img)
    if face_op == None:
        fail_counter+=1
        continue
    _,f,e=face_op
    f=cv2.resize(f,(224,224))[:,:,::-1]
    e=cv2.resize(e,(224,224))[:,:,::-1]
    fea_f=sess.run(conv5_feature,feed_dict={Images:f.reshape((1,224,224,3))})
    fea_e=sess.run(conv5_feature,feed_dict={Images:e.reshape((1,224,224,3))})
    lable_=[0]*16
    lable_[lable_id]=1
    y_cordinate.append(lable_)
    face_matrix.append(fea_f.astype('float32'))
    eye_matrix.append(fea_e.astype('float32'))
    if i%100==0:
        print('pic:{},fail count:{},run time:{}'.format(i,fail_counter,time.time()-s_time))
       # s_time=time.time()
    if (i%50000==0)and (i !=0):
        file_index+=1
        file_name='feature50k_bottle_'+str(file_index)+r'_.pkl'
        fp=open(os.path.join(r'Feature_File_bottleneck2/',file_name),'wb')
        face_matrix=np.array(face_matrix).astype('float32')
        eye_matrix=np.array(eye_matrix).astype('float32')
        y_cordinate=np.array(y_cordinate).astype('uint8')
        pickle.dump(obj={'feature_face':face_matrix,'feature_eye':eye_matrix,'label':y_cordinate},file=fp)
        fp.close()
        y_cordinate=[]
        face_matrix=[]
        eye_matrix=[]
        print('file save')
        #break

In [ ]:
eye_matrix=np.array(eye_matrix).reshape((-1,25088))
face_matrix=np.array(face_matrix).reshape((-1,25088))
y_cordinate=np.array(y_cordinate)

In [ ]:
face_matrix.shape

In [ ]:
fp=open('vgg19_feature.pkl','wb')
pickle.dump(file=fp,obj={'feature_eye':eye_matrix,'feature_face':face_matrix,'label':y_cordinate})
fp.close()

In [ ]:
s=0
face_lis=[]
eye_lis=[]
for i in range(100):
    f_img,e_img=face_matrix[s:s+10],face_matrix[s:s+10]
    fea_f=sess.run(conv5_feature,feed_dict={Images:f_img})
    fea_e=sess.run(conv5_feature,feed_dict={Images:e_img})
    face_lis.append(fea_f)
    eye_lis.append(fea_e)
    print(s)
    s=i+10

In [58]:
a_=np.array([8]*8*8*10).reshape((1,8,8,10))

In [60]:
a=tf.constant(a_)
print(tf.shape(a).eval())

[ 1  8  8 10]


In [69]:
p=tf.nn.max_pool(a,ksize=[1,8,8,1],strides=[1,1,1,1],padding='SAME')
#ps=tf.squeeze(p,[1,2])

In [70]:
p_=sess.run(p)

In [71]:
p_.shape

(1, 8, 8, 10)